# Optimize Doraemon pipeline attributes

In [1]:
import optuna
from stable_baselines3.common.evaluation import evaluate_policy
import gym
from doraemon.doraemon import DoraemonCallback
from env.custom_hopper import *
from PPO.ppo_utils import create_agent, train

# basic RL pipeline attributes
n_trials = 50
n_episodes = 1000
mean_timestep = 100
n_eval_episodes = 10

# optimize ppo attributes
optimized_clip_range = 0.19877024509129543
optimized_learning_rate = 0.0008
optimized_gamma = 0.992


def optimize_doraemon_call(clip_range, learning_rate, gamma,
                           epsilon, step, delta, alpha,
                           n_episodes, n_eval_episodes,
                           env_train="CustomHopper-source-v0",
                           env_test="CustomHopper-target-v0"):
  
   # Re-create agent for evaluation (uses the same env and parameters)
    agent = create_agent(
        env=env_train,
        clip_range=clip_range,
        learning_rate=learning_rate,
        gamma=gamma,
        verbose=0
    )

    train(
        agent, 
        total_timestep=n_episodes,
        callbacks=[
            DoraemonCallback(agent, epsilon=epsilon, step=step, delta=delta, alpha=alpha, verbose=0)
        ]
    )

    # Evaluate on the test env
    test_env = gym.make(env_test)
    mean_reward, _ = evaluate_policy(agent, test_env, n_eval_episodes=n_eval_episodes)

    return mean_reward


def objective(trial):

    # Doraemon-specific parameters
    epsilon = trial.suggest_float("epsilon", 0.01, 0.1, log=True)
    step = trial.suggest_float("step", 0.5, 10.0, log=True)
    delta = trial.suggest_float("delta", 0.1, 0.9, log=True)
    alpha = trial.suggest_float("alpha", 0.1, 0.9, log=True)

    total_reward = optimize_doraemon_call(
        optimized_clip_range,
        optimized_learning_rate,
        optimized_gamma,
        epsilon,
        step,
        delta,
        alpha,
        n_episodes*mean_timestep,
        n_eval_episodes
    )

    return total_reward


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=n_trials, n_jobs=4, show_progress_bar=True)

print("Best parameters:", study.best_params)
print("Best reward:", study.best_value)

/opt/miniconda3/envs/mldl_m4/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-06-16 22:40:09,392] A new study created in memory with name: no-name-e02b8361-1e2f-4b92-8e1a-bce4ebccf52f
  0%|          | 0/50 [00:00<?, ?it/s]/opt/miniconda3/envs/mldl_m4/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
Best trial: 3. Best value: 1330.03:   2%|▏         | 1/50 [06:16<5:07:14, 376.21s/it]

[I 2025-06-16 22:46:25,613] Trial 3 finished with value: 1330.0336747288704 and parameters: {'epsilon': 0.07394508925157602, 'step': 0.8723629186151334, 'delta': 0.8134725616069084, 'alpha': 0.6806439714612296}. Best is trial 3 with value: 1330.0336747288704.


Best trial: 3. Best value: 1330.03:   4%|▍         | 2/50 [06:17<2:04:43, 155.91s/it]

[I 2025-06-16 22:46:27,321] Trial 2 finished with value: 1117.0046644926072 and parameters: {'epsilon': 0.02079833549032097, 'step': 1.3204770764483063, 'delta': 0.2587489448333862, 'alpha': 0.8030002202147642}. Best is trial 3 with value: 1330.0336747288704.


Best trial: 1. Best value: 1339.04:   6%|▌         | 3/50 [06:20<1:07:14, 85.85s/it] 

...

[I 2025-06-17 00:00:22,264] Trial 47 finished with value: 675.4586905121803 and parameters: {'epsilon': 0.018865125105137912, 'step': 2.752429791687192, 'delta': 0.20745852011466115, 'alpha': 0.48985325743538105}. Best is trial 18 with value: 1652.352514833212.


Best trial: 18. Best value: 1652.35:  98%|█████████▊| 49/50 [1:22:16<01:21, 81.99s/it]

[I 2025-06-17 00:02:26,063] Trial 48 finished with value: 1142.5502455115318 and parameters: {'epsilon': 0.019647682154118915, 'step': 5.2859065324328345, 'delta': 0.2202191255211601, 'alpha': 0.311214966298559}. Best is trial 18 with value: 1652.352514833212.


Best trial: 18. Best value: 1652.35: 100%|██████████| 50/50 [1:22:17<00:00, 98.76s/it]

[I 2025-06-17 00:02:27,173] Trial 49 finished with value: 1226.3171590983868 and parameters: {'epsilon': 0.019703037167380637, 'step': 5.178305582279415, 'delta': 0.23309801375934935, 'alpha': 0.4983287516825577}. Best is trial 18 with value: 1652.352514833212.
Best parameters: {'epsilon': 0.09168764322520681, 'step': 2.502828879059888, 'delta': 0.17954957384916176, 'alpha': 0.5429804944757511}
Best reward: 1652.352514833212
